In [ ]:
%load_ext autoreload
%autoreload 2
from data.input_data import get_all_data
from pyomo.environ import *
from model.sets_params import define_sets_and_params
from model.variables import define_variables
from model.objective import define_objective
import model.affichage as affichage
# from model.solver_GLPK import solve_model
# from model.solver_CBC import solve_model
from model.solver_scip import solve_model
# from model.solver_scip_FAST import solve_model
# from model.solver_scip_STRONG import solve_model
from model.constraints_copy import add_constraints
# 1. Chargement des données
data = get_all_data()
print("Données chargées avec succès.")

Saving matrix Distor_ihc with dimensions 20x5x4
Données chargées avec succès.


In [2]:
# 2. Création du modèle Pyomo
model = ConcreteModel()
print("Modèle Pyomo créé.")
model = define_sets_and_params(model, data)
print("Ensembles et paramètres définis.")
define_variables(model)
print("Variables définies.")
model = add_constraints(model, data)  # ou juste add_constraints(model, data)
print("Contraintes ajoutées.")
define_objective(model)
print("Objectif défini.")
# 3. Affichage des informations du modèle
num_vars = sum(1 for v in model.component_objects(Var, active=True) for _ in v)
num_constraints = sum(1 for c in model.component_objects(Constraint, active=True) for _ in c)

print(f"Nombre total de variables     : {num_vars}")
print(f"Nombre total de contraintes  : {num_constraints}")

# Variables binaires
num_bin_vars = sum(1 for v in model.component_objects(Var, active=True)
                for index in v if v[index].domain == Binary)
print(f"Nombre de variables binaires : {num_bin_vars}")

# Variables continues
num_cont_vars = sum(1 for v in model.component_objects(Var, active=True)
                    for index in v if v[index].domain == NonNegativeReals)
print(f"Nombre de variables continues: {num_cont_vars}")

# 3. Résolution
result = solve_model(model, tee=True)
model.write('modele.lp', io_options={'symbolic_solver_labels': True})
    
# print("\n------ QUALITÉ DES COMPOSANTS (ratio) ---------")       
# for c in model.C:
#     for k in list(model.K)[:2]:
#         mu = model.mu_ck[c, k].value or 0
#         D = model.D_k[k]
#         ratio = mu / D if D > 0 else 0
#         j = model.lamda_k[k]
#         bmin = model.BetaMin_cj[j, c]
#         bmax = model.BetaMax_cj[j, c]
#         print(f"c={c}, k={k}: ratio={ratio:.4f}  [{bmin}, {bmax}]")

Modèle Pyomo créé.
Ensembles et paramètres définis.
Variables définies.
Contraintes ajoutées.
Objectif défini.
Nombre total de variables     : 28916
Nombre total de contraintes  : 57962
Nombre de variables binaires : 14112
Nombre de variables continues: 14804
✅ SCIP disponible. Lancement de la résolution avec gap = 0.15...
SCIP version 9.0.1 [precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex 7.0.1] [GitHash: bebb64304e]
Copyright (c) 2002-2024 Zuse Institute Berlin (ZIB)

External libraries: 
  Soplex 7.0.1         Linear Programming Solver developed at Zuse Institute Berlin (soplex.zib.de) [GitHash: 1cc71921]
  CppAD 20180000.0     Algorithmic Differentiation of C++ algorithms developed by B. Bell (github.com/coin-or/CppAD)
  MPIR 3.0.0           Multiple Precision Integers and Rationals Library developed by W. Hart (mpir.org)
  ZIMPL 3.6.0          Zuse Institute Mathematical Programming Language developed by T. Koch (zimpl.zib.de)
  AMPL/MP 690e9e7      AMPL .n

('modele.lp', 2250716534544)

In [ ]:
df = affichage.extract_blending_results(model)
affichage.save_blending_results_to_csv(df)
affichage.export_fusion_style_table(df, filename="S_blending_table.png")

✅ Résultats enregistrés dans blending_results_table15.csv
✅ Tableau exporté dans : S_blending_table.png


In [19]:
df.head(10)

,N°,Commande,Qualité demandée (QM),Nom ingrédient,Stock source,Gamme,t,x_ihkt (tonnes),Real_X_ihkt,o_ihkt (binaire),E_k,L_k,Quantity commande,Total_commande,%_blending
9,1,Commande 1,Profil_BG,BT_Smine,La mine,traitement TM (2),1,6787.648347,6108.883512,1.0,1,4,12000,12000.0,56.563736
55,1,Commande 1,Profil_BG,BT_Sc2,Stock prés laverie,Laverie (4),1,1738.008451,1564.207605,1.0,1,4,12000,12000.0,14.483404
25,1,Commande 1,Profil_BG,BT_Smine,La mine,TM + Laverie (3),1,1198.975639,959.180511,1.0,1,4,12000,12000.0,9.991464
10,1,Commande 1,Profil_BG,BT_Smine,La mine,traitement TM (2),3,3741.920413,3367.728371,1.0,1,4,12000,12000.0,31.182670
52,2,Commande 2,Export,BT_Sc1,Stock criblé 1,Laverie (4),1,2149.401222,1934.461100,1.0,3,6,12000,12000.0,17.911677
40,2,Commande 2,Export,MT_Smine,La mine,traitement TM (2),2,4500.000000,4050.000000,1.0,3,6,12000,12000.0,37.500000
71,2,Commande 2,Export,MT_Sf1,Stock lavé,sans traitement (0),2,1557.777778,1557.777778,1.0,3,6,12000,12000.0,12.981481
53,2,Commande 2,Export,MT_Sc1,Stock criblé 1,sans traitement (0),2,1100.000000,1100.000000,1.0,3,6,12000,12000.0,9.166667
49,2,Commande 2,Export,BT_Sc1,Stock criblé 1,sans traitement (0),2,615.803450,615.803450,1.0,3,6,12000,12000.0,5.131695
41,2,Commande 2,Export,MT_Smine,La mine,traitement TM (2),4,3046.619636,2741.957672,1.0,3,6,12000,12000.0,25.388497


In [ ]:
df_t = affichage.extract_tenurs_results(model)
df_t.head()

,Commande,Qualité demandée (QM),mu_ck,D_k,Cible_BPL,Teneur_BPL,Cible_MgO,Teneur_MgO,Cible_SiO2,Teneur_SiO2,Cible_Cd,Teneur_Cd
0,Commande 1,Profil_BG,9.225836e+05,12000,58.995,58.995,1.0,0.886965,10.0,8.000000,7.32375,9.000
1,Commande 2,Export,1.022299e+06,12000,65.500,65.500,0.6,0.756600,8.0,8.000000,10.93500,10.935
2,Commande 3,Tess,9.252752e+05,12000,58.995,58.995,1.0,0.884250,10.0,8.227016,5.40000,9.000
3,Commande 4,BG_BT,9.364921e+05,12000,58.500,58.500,0.6,0.741012,8.0,8.000000,10.80000,10.800
4,Commande 5,Stand,9.237334e+05,12000,58.995,58.995,1.0,0.900000,10.0,8.082785,6.48000,9.000


In [20]:
# df.head(50)  # Affichage des premières lignes du DataFrame

In [ ]:
# affichage.export_fusion_style_table(df, filename="S_blending_table.png")

✅ Tableau exporté dans : S_blending_table.png


In [21]:
# df.head(10)  # Affiche les premières lignes du DataFrame pour vérification

In [22]:
import pandas as pd

def extract_final_stocks(model):
    i_names = model.I_names
    stock_sources = model.Stock_sources
    T_max = max(model.T)  # Dernière période

    results = []
    for i in model.I:
        val = model.S_it[i, T_max].value
        results.append({
            "Nom ingrédient": i_names[i],
            "Stock source": stock_sources[i],
            f"Stock final à t={T_max} (tonnes)": val
        })
    df = pd.DataFrame(results)
    return df

def save_final_stocks_to_csv(df, filename='stock_final.csv'):
    df.to_csv(filename, index=False)
    print(f"✅ Stock final enregistré dans {filename}")

def show_final_stocks(df):
    print("\n=== Tableau des stocks finaux ===")
    print(df)

# Utilisation typique (dans main.py ou un notebook)
df_stock = extract_final_stocks(model)
save_final_stocks_to_csv(df_stock)
show_final_stocks(df_stock)


✅ Stock final enregistré dans stock_final.csv

=== Tableau des stocks finaux ===
   Nom ingrédient        Stock source  Stock final à t=28 (tonnes)
0       TBT_Smine             La mine                 1.205473e+06
1        BT_Smine             La mine                 2.345952e+04
2       PBG_Smine             La mine                 8.936755e+04
3        MT_Smine             La mine                 2.642291e+04
4         TBT_Sc1      Stock criblé 1                 3.000000e+03
5          BT_Sc1      Stock criblé 1                 0.000000e+00
6         PBG_Sc1      Stock criblé 1                 2.140000e+04
7          MT_Sc1      Stock criblé 1                 0.000000e+00
8         TBT_Sc2  Stock prés laverie                 1.330000e+04
9          BT_Sc2  Stock prés laverie                 0.000000e+00
10        PBG_Sc2  Stock prés laverie                 6.500000e+03
11         MT_Sc2  Stock prés laverie                 0.000000e+00
12         TBT_Se       Stock épierré           

In [11]:
# def diagnostic_flux_mine(model):
#     print("===== DIAGNOSTIC FLUX MINE =====")
#     for i in model.QS_mines:
#         total = 0
#         for h in model.H:
#             for k in model.K:
#                 for t in model.T:
#                     if (i, h, k, t) in model.IHKT_valid:
#                         val = model.x_ihkt[i, h, k, t].value
#                         if val is not None:
#                             total += val
#         print(f"Ingrédient {i} (Mine): Total extrait sur tout l’horizon = {total}")
#         print(f"Stock initial = {model.Stock_initial_i[i]}")
# diagnostic_flux_mine(model)

In [23]:
# 1. Demande totale
demande_totale = sum([model.D_k[k] for k in model.K])
print(f"Demande totale sur tout l'horizon (somme D_k) : {demande_totale:.0f} tonnes")

# 2. Stock initial total
stock_initial_total = sum([model.Stock_initial_i[i] for i in model.I])
print(f"Stock initial total (somme Stock_initial_i) : {stock_initial_total:.0f} tonnes")

# 3. Affiche le ratio
if demande_totale > stock_initial_total:
    print(f"\n⚠️  La demande ({demande_totale:.0f}) est supérieure au stock initial ({stock_initial_total:.0f}) !")
else:
    print(f"\nLa demande ({demande_totale:.0f}) est couverte par le stock initial ({stock_initial_total:.0f})")


Demande totale sur tout l'horizon (somme D_k) : 168000 tonnes
Stock initial total (somme Stock_initial_i) : 1242800 tonnes

La demande (168000) est couverte par le stock initial (1242800)


In [24]:
# Somme des stocks initiaux pour QS_Sc1
stock_QS_Sc1 = sum([model.Stock_initial_i[i] for i in model.QS_Sc1])
print(f"Stock initial total dans QS_Sc1 : {stock_QS_Sc1:.0f} tonnes")

# Somme des stocks initiaux pour QS_Se
stock_QS_Se = sum([model.Stock_initial_i[i] for i in model.QS_Se])
print(f"Stock initial total dans QS_Se  : {stock_QS_Se:.0f} tonnes")

# Somme des stocks initiaux pour QSL_Sf1
stock_QSL_Sf1 = sum([model.Stock_initial_i[i] for i in model.QSL_Sf1])
print(f"Stock initial total dans QSL_Sf1 : {stock_QSL_Sf1:.0f} tonnes")

# Optionnel : total cumulé sur les trois
total_stocks = stock_QS_Sc1 + stock_QS_Se + stock_QSL_Sf1
print(f"Somme totale des stocks initiaux utilisés (QS_Sc1 + QS_Se + QSL_Sf1) : {total_stocks:.0f} tonnes")

# 1. Demande totale
demande_totale = sum([model.D_k[k] for k in model.K])
print(f"Demande totale sur tout l'horizon (somme D_k) : {demande_totale:.0f} tonnes")
# 3. Affiche le ratio
if demande_totale > stock_initial_total:
    print(f"\n⚠️  La demande ({demande_totale:.0f}) est supérieure au stock initial ({total_stocks:.0f}) !")
else:
    print(f"\nLa demande ({demande_totale:.0f}) est couverte par le stock initial ({total_stocks:.0f})")



Stock initial total dans QS_Sc1 : 31000 tonnes
Stock initial total dans QS_Se  : 20300 tonnes
Stock initial total dans QSL_Sf1 : 16000 tonnes
Somme totale des stocks initiaux utilisés (QS_Sc1 + QS_Se + QSL_Sf1) : 67300 tonnes
Demande totale sur tout l'horizon (somme D_k) : 168000 tonnes

La demande (168000) est couverte par le stock initial (67300)


In [25]:
print("===== CONTRÔLE DES FLUX D’UTILISATION =====")
for i in model.I:
    initial = model.Stock_initial_i[i]
    extrait = sum(model.x_ihkt[i, h, k, t].value for h in model.H for k in model.K for t in model.T if (i, h, k, t) in model.IHKT_valid)
    final = model.S_it[i, model.T.last()].value
    print(f"Ingrédient {i:2} | Stock initial: {initial:8.0f} | Extrait total: {extrait:8.0f} | Stock final: {final:8.0f} | Flux total = {initial + sum(model.Ait[t, i] for t in model.T) - final:8.0f}")


===== CONTRÔLE DES FLUX D’UTILISATION =====
Ingrédient  1 | Stock initial:  1113900 | Extrait total:    20427 | Stock final:  1205473 | Flux total =    20427
Ingrédient  2 | Stock initial:     5000 | Extrait total:    65540 | Stock final:    23460 | Flux total =    65540
Ingrédient  3 | Stock initial:    21400 | Extrait total:    16032 | Stock final:    89368 | Flux total =    16032
Ingrédient  4 | Stock initial:     4600 | Extrait total:    34177 | Stock final:    26423 | Flux total =    34177
Ingrédient  5 | Stock initial:     3000 | Extrait total:        0 | Stock final:     3000 | Flux total =        0
Ingrédient  6 | Stock initial:     5000 | Extrait total:     5000 | Stock final:        0 | Flux total =     5000
Ingrédient  7 | Stock initial:    21400 | Extrait total:        0 | Stock final:    21400 | Flux total =        0
Ingrédient  8 | Stock initial:     1600 | Extrait total:     1600 | Stock final:        0 | Flux total =     1600
Ingrédient  9 | Stock initial:    13300 | Ex

In [38]:
import pandas as pd
import model.affichage as affichage


In [42]:
df_flux = affichage.Stocks_utilisation(model)
df_flux.head(5) 
# df_flux.tail()

,Ingrédient,Stock initial,Extrait total,Ajout total,Flux Sortant,Flux Entrant,Stock final
0,TBT_Smine (1),1113900,20427.23,112000,1093472.77,112000,1205472.77
1,BT_Smine (2),5000,65540.48,84000,-60540.48,84000,23459.52
2,PBG_Smine (3),21400,16032.45,84000,5367.55,84000,89367.55
3,MT_Smine (4),4600,34177.09,56000,-29577.09,56000,26422.91
4,TBT_Sc1 (5),3000,0.00,0,3000.00,0,3000.00


In [51]:

df_util = affichage.compute_resource_utilization(model, data)
# df_util.to_csv("resource_utilization.csv", index=False)
print(df_util.shape)  # aperçu
df_util.head(20) # aperçu



(140, 5)


,Période,Ressource,Flux_total,Capacité,Taux_utilisation
0,1,1,4500.000000,4500.0,1.000000
1,1,2,0.000000,3600.0,0.000000
2,1,3,0.000000,3600.0,0.000000
3,1,4,3273.581228,4750.0,0.689175
4,1,5,7323.581228,7600.0,0.963629
5,2,1,4500.000000,4500.0,1.000000
6,2,2,0.000000,3600.0,0.000000
7,2,3,758.079587,3600.0,0.210578
8,2,4,606.463670,4750.0,0.127677
9,2,5,3974.192041,7600.0,0.522920


In [16]:
df.head(50)

,N°,Commande,Qualité demandée (QM),Nom ingrédient,Stock source,Gamme,t,x_ihkt (tonnes),Real_X_ihkt,o_ihkt (binaire),E_k,L_k,Quantity commande,Total_commande,%_blending
9,1,Commande 1,Profil_BG,BT_Smine,La mine,traitement TM (2),1,6787.648347,6108.883512,1.0,1,4,12000,12000.0,56.563736
55,1,Commande 1,Profil_BG,BT_Sc2,Stock prés laverie,Laverie (4),1,1738.008451,1564.207605,1.0,1,4,12000,12000.0,14.483404
25,1,Commande 1,Profil_BG,BT_Smine,La mine,TM + Laverie (3),1,1198.975639,959.180511,1.0,1,4,12000,12000.0,9.991464
10,1,Commande 1,Profil_BG,BT_Smine,La mine,traitement TM (2),3,3741.920413,3367.728371,1.0,1,4,12000,12000.0,31.182670
52,2,Commande 2,Export,BT_Sc1,Stock criblé 1,Laverie (4),1,2149.401222,1934.461100,1.0,3,6,12000,12000.0,17.911677
40,2,Commande 2,Export,MT_Smine,La mine,traitement TM (2),2,4500.000000,4050.000000,1.0,3,6,12000,12000.0,37.500000
71,2,Commande 2,Export,MT_Sf1,Stock lavé,sans traitement (0),2,1557.777778,1557.777778,1.0,3,6,12000,12000.0,12.981481
53,2,Commande 2,Export,MT_Sc1,Stock criblé 1,sans traitement (0),2,1100.000000,1100.000000,1.0,3,6,12000,12000.0,9.166667
49,2,Commande 2,Export,BT_Sc1,Stock criblé 1,sans traitement (0),2,615.803450,615.803450,1.0,3,6,12000,12000.0,5.131695
41,2,Commande 2,Export,MT_Smine,La mine,traitement TM (2),4,3046.619636,2741.957672,1.0,3,6,12000,12000.0,25.388497


In [17]:
# %load_ext autoreload
# %autoreload 2
# from data.input_data import get_all_data
# from pyomo.environ import *
# from model.sets_params import define_sets_and_params
# from model.variables import define_variables
# from model.objective import define_objective
# # from model.solver_GLPK import solve_model
# # from model.solver_CBC import solve_model
# from model.solver_scip import solve_model
# from model.constraints_copy import add_constraints
# # 1. Chargement des données
# data = get_all_data()
# # 2. Création du modèle Pyomo
# model = ConcreteModel()
# model = define_sets_and_params(model, data)
# define_variables(model)
# model = add_constraints(model, data)  # ou juste add_constraints(model, data)
# define_objective(model)

In [18]:
# model.test=Param(model.T, initialize=0)  # Exemple d'initialisation